In [1]:
## Here I copied the code used in the first two code cells in homework 2 to install BeautifulSoup, etc.
import sys
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} bs4
import requests #package for http requests
import bs4 # package for html parsing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [23]:
def web_to_soup(web_address, file_name):
    response = requests.get(web_address)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    file = file_name
    with open(file, mode='w', encoding='utf-8') as f:
        f.write(response.text)
    return soup

In [25]:
def extract_categories(soup):
    """ returns a list of categories from the left side of the samhsa data tables. """
    search = soup.find_all("th",{"class":["subhead2","subhead1"]}) 
    categories = []
    for tag in search:
        categories.append(tag.text)
    categories = np.array(categories)
    return categories

In [26]:
## pull out the data for each demographic category 
def pull_out_data(soup):
    search = soup.find_all("td") 
    data_entries = []
    for tag in search:
        data_entries.append(tag.text)
    data_entries = np.array(data_entries)
    return data_entries

In [27]:
def delete_blank(data):
    """go through the array entries and delete them if they're blank using a for-loop
    data is a numpy array"""
    to_delete = []
    for i in range(len(data)):
        if data[i]=='\xa0':
            to_delete.append(i)
    data = np.delete(data, to_delete)
    return data

In [28]:
def delete_footnotes(data_array):
    for i in range(len(data_array)):
        if('a' in data_array[i]):
            length = len(data_array[i])
            data_array[i]=data_array[i][:-1]
    return data_array

In [29]:
years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

In [30]:
def arrange_data(array, number_of_rows, number_of_columns):
    """ puts data in rows and columns """
    formatted_data=[]
    count=0
    for i in range(number_of_rows):
        formatted_data.append(array[count:count+number_of_columns])
    count+=number_of_columns
    return formatted_data

In [31]:
soup1=web_to_soup('https://www.samhsa.gov/data/sites/default/files/reports/rpt29394/NSDUHDetailedTabs2019/NSDUHDetTabsSect10pe2019.htm', 'samhsa1.html')

## a bit of formatting/editing for the first table
demographics1 = extract_categories(soup1)
demographics1 = demographics1[:14]
demographics1 = np.insert(demographics1,0,"total")

data1=pull_out_data(soup1)
data1=data1[:216]

data1=delete_blank(data1)

data1 = delete_footnotes(data1)

data1_formatted = arrange_data(data1, 15, 12)

df = pd.DataFrame(formatted_data, demographics1, years)
df.head(15)


,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
total,"39,826","41,195","41,417","41,381","43,714","43,846","43,553","43,421","44,652","46,632","47,635","51,495"
18-25,"6,099","6,043","6,151","6,356","6,774","6,754","7,014","7,574","7,635","8,849","8,944","9,930"
26 or Older,"33,727","35,152","35,266","35,025","36,940","37,092","36,539","35,847","37,016","37,783","38,691","41,565"
26-49,"20,697","21,455","20,648","19,936","20,777","21,061","20,007","20,589","20,903","22,256","22,721","25,279"
50 or Older,"13,030","13,698","14,618","15,089","16,163","16,031","16,531","15,257","16,114","15,527","15,969","16,286"
Male,"14,778","15,614","16,461","15,895","16,856","16,452","16,371","16,785","17,142","18,065","18,308","19,758"
Female,"25,048","25,581","24,956","25,486","26,858","27,394","27,182","26,636","27,510","28,567","29,327","31,737"
Not Hispanic or Latino,"34,971","36,321","36,580","36,793","38,049","37,828","37,795","37,919","38,598","40,603","40,785","44,055"
White,"28,286","29,799","29,704","29,552","30,066","30,222","30,074","30,387","31,266","32,183","32,210","34,967"
Black or African American,"4,314","4,113","4,520","4,364","5,044","4,688","4,603","4,410","4,171","4,782","4,802","5,174"
